In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
65,ABW,Aruba,2020-05-27,101,0,3,0,945.994,0.000,28.099,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
204,AFG,Afghanistan,2020-05-27,11831,658,220,1,303.917,16.903,5.651,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
271,AGO,Angola,2020-05-27,71,2,4,0,2.160,0.061,0.122,...,2.405,1.362,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-26,France,145457.0,0.0,28486.0,0.0,0.0,0.0
0,2020-05-26,China,84171.0,0.0,4655.0,0.0,0.0,0.0
0,2020-05-26,Italy,230789.0,0.0,32945.0,0.0,0.0,0.0
0,2020-05-26,Spain,235879.0,0.0,26614.0,0.0,0.0,0.0
0,2020-05-26,United States,1684896.0,0.0,98762.0,0.0,0.0,0.0
0,2020-05-26,World,5568119.0,0.0,347524.0,0.0,0.0,0.0
0,2020-05-26,United Kingdom,263885.0,0.0,37078.0,0.0,0.0,0.0
0,2020-05-26,Germany,179368.0,0.0,8317.0,0.0,0.0,0.0
0,2020-05-26,Iran,139969.0,0.0,7516.0,0.0,0.0,0.0
0,2020-05-26,Turkey,158885.0,0.0,4398.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-27,France,145457.0,145555.0,28486.0,28530.0,-98.0,-44.0
0,2020-05-27,China,84171.0,84103.0,4655.0,4638.0,68.0,17.0
0,2020-05-27,Italy,230789.0,230555.0,32945.0,32955.0,234.0,-10.0
0,2020-05-26,Spain,235879.0,236259.0,26614.0,27117.0,-380.0,-503.0
0,2020-05-27,United States,1684896.0,1681212.0,98762.0,98916.0,3684.0,-154.0
0,2020-05-27,World,5568119.0,5555708.0,347524.0,350212.0,12411.0,-2688.0
0,2020-05-27,United Kingdom,263885.0,265227.0,37078.0,37048.0,-1342.0,30.0
0,2020-05-27,Germany,179368.0,179364.0,8317.0,8349.0,4.0,-32.0
0,2020-05-27,Iran,139969.0,139511.0,7516.0,7508.0,458.0,8.0
0,2020-05-27,Turkey,158885.0,158762.0,4398.0,4397.0,123.0,1.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")